#### 导入工具包

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from math import log
from collections import Counter

#### 创建数据集

In [2]:
def create_data():
    # 创建数据集
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    # 给出数据集的列名称
    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    
    return datasets, labels

In [3]:
datasets, labels = create_data()
train_data = pd.DataFrame(datasets, columns=labels)

#### 计算信息增益

#####  计算信息熵

In [4]:
def calc_ent(dataframe, c):
    target = dataframe.iloc[:,c]
    data_length = len(target)
    labels = Counter(target)
    ent = -sum([(p/data_length)*log(p/data_length, 2) for p in list(labels.values())] )
    return ent

##### 计算特征拆分之后的信息熵

In [5]:
def cond_ent(dataframe, index=0):
    data_length = len(dataframe)
    feature_set = {}
    data_group = dataframe.groupby(dataframe.iloc[:, index])
    for group in data_group:
        key = group[0]
        val = group[1]
        feature_set[key] = val
    
    cond_ent = sum([(len(p)/data_length)*calc_ent(p) for p in feature_set.values()])
    return cond_ent

##### 计算信息增益率|

In [6]:
def info_gain_ratio(ent, cond_ent, iv_ent):
    return (ent - cond_ent)/iv_ent

##### 计算信息增益率最大的特征

In [7]:
def info_gain_train(dataframe):
    count = dataframe.shape[1] - 1
    ent = calc_ent(dataframe)
    best_feature = []
    for c in range(count):
        c_info_gain = info_gain_ratio(calc_ent(dataframe, -1), cond_ent(dataframe, index=c), calc_ent(dataframe, c))
        best_feature.append((c, c_info_gain))
        print('特征({}) - info_gain_ratio - {:.3f}'.format(labels[c], c_info_gain))
    best_ = max(best_feature, key=lambda x: x[-1])
    return '特征({})的信息增益率最大，选择为根节点特征'.format(labels[best_[0]])

#### 使用类组织代码

##### 构造树结构

In [8]:
class Node:
    def __init__(self, root=True, label=None, feature_name=None, feature=None):
        # 节点处理思路
        """
        1. 对于叶子节点，返回该节点对应的所有的样本标签值
        2. 对于中间节点，返回各个特征拆分后对应的子节，并添加到树结构中
        """
        self.root = root               # 当前节点是否是叶子节点
        self.label = label             # 如果当前节点是是叶子节点，则返回该节点对应的预测值
        self.feature_name = feature_name        # 用作切分的特征的名称
        self.feature = feature                  # 用作切分的特征对应的索引值
        self.tree = {}                          # 树结构
        self.result = {
            'label': self.label,
            'feature': self.feature,
            'tree': self.tree
        }
        
    def __repr__(self):
        return '{result}'.format(result = self.result)
    
    # 训练过程中使用
    def add_node(self, val, node):
        self.tree[val] = node
    
    # 预测过程中使用
    def predict(self, features):
        if self.root is True:
            return self.label
        return self.tree[features[self.feature]].predict(features)

##### 填充树结构

In [9]:
class DTree:
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon           # 超参数，当信息增益小于某个值的时候不再继续分裂
        self._tree = {}
        
    # 计算信息熵
    @staticmethod
    def calc_ent(dataframe, c=-1):
        target = dataframe.iloc[:,c]
        data_length = len(target)
        labels = Counter(target)
        ent = -sum([(p/data_length)*log(p/data_length, 2) for p in list(labels.values())] )
        return ent
    
    # 计算拆分后特征的信息熵
    def cond_ent(self, dataframe, index=0):
        data_length = len(dataframe)
        feature_set = {}
        data_group = dataframe.groupby(dataframe.iloc[:, index])
        for group in data_group:
            key = group[0]
            val = group[1]
            feature_set[key] = val

        cond_ent = sum([(len(p)/data_length)*self.calc_ent(p) for p in feature_set.values()])
        return cond_ent
    
    # 计算信息增益率
    @staticmethod
    def info_gain_ratio(ent, cond_ent, iv_ent):
        return (ent - cond_ent)/iv_ent
    
    # 返回信息增益最大的特征
    def info_gain_train(self, dataframe):
        count = dataframe.shape[1] - 1
        ent = self.calc_ent(dataframe)
        best_feature = []
        for c in range(count):
            c_info_gain = self.info_gain_ratio(self.calc_ent(dataframe, -1), self.cond_ent(dataframe, index=c), self.calc_ent(dataframe, c))
            best_feature.append((c, c_info_gain))
        # 比较大小
        best_ = max(best_feature, key=lambda x: x[-1])
        return best_
    
    # 定义训练部分
    def train(self, train_data):
        y_train, features = train_data.iloc[:, -1], list(train_data.columns)[:-1]
        
        """设置边界条件"""
        # 1.如果节点中的标签属于同一类别，则当前节点为叶子节点
        if len(y_train.value_counts()) == 1:
            return Node(root=True, label=y_train.iloc[0])
        
        # 2.如果拆分到最后，没有可以继续拆分的特征了，则将当前节点中数量最大的的类别作为最终的类别
        if len(features) == 0:
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])
        
        # 3. 如果当前节点继续拆分之后的信息增益小于阈值，则停止拆分
        max_feature, max_info_gain = self.info_gain_train(train_data)
        max_feature_name = features[max_feature]
        if max_info_gain < self.epsilon:
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])
        
        """如果当前节点不满足成为叶子节点的条件，则成为分枝节点，继续拆分"""
        node_tree = Node(root=False, feature_name=max_feature_name, feature=max_feature)
        feature_list = train_data[max_feature_name].value_counts().index
        # 4. 递归生成树
        for f in feature_list:
            sub_train_df = train_data.loc[train_data[max_feature_name]==f].drop(max_feature_name, axis=1)
            sub_tree = self.train(sub_train_df)
            node_tree.add_node(f, sub_tree)
        
        return node_tree
    
    def fit(self, train_data):
        self._tree = self.train(train_data)
        return self._tree
    
    def predict(self, X_test):
        return self._tree.predict(X_test)

##### 运行并查看结果

In [10]:
# 创建数据集
datasets, labels = create_data()
data_df = pd.DataFrame(datasets, columns=labels)

# 创建树
dt = DTree()
tree = dt.fit(data_df)

# 查看树结构
print(tree)

# 预测数据
dt.predict(['老年', '否', '否', '一般'])

{'label': None, 'feature': 2, 'tree': {'否': {'label': None, 'feature': 1, 'tree': {'否': {'label': '否', 'feature': None, 'tree': {}}, '是': {'label': '是', 'feature': None, 'tree': {}}}}, '是': {'label': '是', 'feature': None, 'tree': {}}}}


'否'